In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

OPENAI_PROXY = "https://ip"

llm = ChatOpenAI(base_url=OPENAI_PROXY, api_key="any", model="gpt-4o", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [ ]:
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

In [ ]:
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

In [ ]:
agent_executor.invoke({"input": "Describe the schema of the playlist table"})

In [ ]:
agent_executor.invoke({"input": "Describe the playlists table"})

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)

In [ ]:
agent_executor.invoke({"input": "Who are the top 3 best selling artists?"})